<a href="https://www.kaggle.com/code/dmid2qwde/model?scriptVersionId=99141528" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install yfinance

  Preparing metadata (setup.py) ... - done
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=30b04e9dbfd5853af9bac19c08cf0ecffc8b76c7fa9fbd505f2b4a1266c80152
  Stored in directory: /root/.cache/pip/wheels/34/ba/79/c0260c6f1a03f420ec7673eff9981778f293b9107974679e36
Successfully built multitasking


In [2]:
!pip install prophet

In [3]:
# essentials
import numpy as np 
import pandas as pd 
import math
import datetime
from datetime import datetime as dt
import yfinance as yf
# visualizations
import seaborn as sns

# time series algorithm
from prophet import Prophet
# reproducibility
np.random.seed(34)

# Jupyter magic

In [4]:
def get_data():
    data_usdrub = yf.download('USDRUB=X', start='2017-01-01', end=datetime.datetime.now().date())
    data_eurub = yf.download('EURRUB=X', start='2017-01-01', end=datetime.datetime.now().date())
    return data_usdrub,data_eurub

In [5]:
def ml_model(df):
    df_columns = ['ds','y']
    train_columns = ['Close']
    train = df[train_columns]
    train = train.rename(columns={'Close':'y'})
    train.index.names =['ds']
    train =train.reset_index()
    df = df.rename(columns={'Close':'y'})
    df.index.names =['ds']
    df =train.reset_index()
    
    m = Prophet(daily_seasonality=False,yearly_seasonality=True,
            weekly_seasonality= True,changepoint_prior_scale= 0.05, 
            seasonality_prior_scale= 20.0, holidays_prior_scale= 5.0)
    m.fit(train)
    
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)
    
    cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df[df_columns].set_index('ds'))
    cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
    mae=np.mean(abs(cmp_df[-45:-15]['e']))
    
    forecast = forecast[['ds','yhat_lower']][-30:]
    forecast['yhat_lower'] = forecast['yhat_lower'].apply(lambda x:x-0.9*mae)
    return forecast
    
    

In [6]:
def get_forecast(df):
    forecast = ml_model(df)
    forecast = float(forecast[['yhat_lower']][-30:-29].iloc[0])
    return forecast

In [7]:
def ml_predict(usd=True,eur=True):
    df_usdrub,df_eurub=get_data()
    if usd==True:
        forecast_usdrub = get_forecast(df_usdrub)
        return forecast_usdrub
    if eur==True:
        forecast_eurub = get_forecast(df_eurub)
        return forecast_eurub
        
    

In [8]:
usdp = ml_predict(eur=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Initial log joint probability = -9.77645
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 136)

Error evaluating model log probability: Non-finite gradient.
Error evaluating model log probability: Non-finite gradient.

      99       3860.73   0.000158984       147.432      0.1836      0.1836      143   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     140       3863.23   9.32079e-05       133.923   7.583e-07       0.001      243  LS failed, Hessian reset 
     199       3864.11    0.00125307       157.161      0.9077      0.9077      317   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     209        3864.4   7.88128e-0

In [9]:
usdp

52.436157230565684

In [10]:
eurp =  ml_predict(usd=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Initial log joint probability = -17.1519
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3590.71     0.0133976       188.263           1           1      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     161       3594.54   0.000101735       114.469   8.838e-07       0.001      242  LS failed, Hessian reset 
     199       3597.15    0.00110145       115.415      0.2679      0.5795      289   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3598.91   0.000639047       68.1616      0.7854      0.7854      414   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     321       3599.38   9.35518e-05       98.9996   6.826e-07       0.001      4

In [11]:
eurp

55.608719403242866